In [5]:
from sklearn.metrics import mean_squared_error
import numpy as np

def evaluate_models(dic_models, X_train_val, y_train_val, X_test_val, y_test_val):
    """
    Treina e avalia uma lista de modelos com base no RMSE.
    
    Parâmetros:
    - models: lista de modelos sklearn (instâncias já inicializadas)
    - X_train_val, y_train_val: dados de treino/validação
    - X_test_val, y_test_val: dados de teste
    
    Retorna:
    - Um dicionário com os RMSE de cada modelo
    """
    
    for name, model in dic_models.values():
        model.fit(X_train_val, y_train_val)  # Treina o modelo
        y_pred = model.predict(X_test_val)   # Faz previsões
        rmse = np.sqrt(mean_squared_error(y_test_val, y_pred))  # Calcula RMSE
        
        model_name = name
        print(f'{model_name} RMSE: {rmse:.2f}')

In [7]:
from sklearn.model_selection import cross_val_score
import numpy as np

def cross_validate_models(dic_models, X_train, y_train, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1):
    """
    Executa cross-validation para uma lista de modelos e retorna o RMSE médio de cada um.

    Parâmetros:
    - models: lista de modelos sklearn (instâncias já inicializadas)
    - X_train, y_train: dados de treino
    - cv: número de folds na validação cruzada (padrão: 3)
    - scoring: métrica de avaliação (padrão: 'neg_root_mean_squared_error' para RMSE)
    - n_jobs: número de threads a serem usadas (-1 usa todos os núcleos disponíveis)

    Retorna:
    - Um dicionário com a média do RMSE para cada modelo
    """

    for name, model in dic_models.values():
        
        scores = cross_val_score(
            model,
            X_train,
            y_train,
            cv=cv, 
            scoring=scoring, 
            n_jobs=n_jobs)
        
        scores = -scores  # Converte para valores positivos, pois a métrica vem negativa

        model_name = name
        print(f'{model_name} CV RMSE: {scores}, mean : {scores.mean():.2f}')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score,
    precision_recall_curve, roc_curve, auc
)
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

def evaluate_classification_models(dic_models, X_train, y_train, cv=3, n_jobs=-1):
    """
    Avalia modelos de classificação binária com métricas diversas.
    
    Parâmetros:
    - models: lista de modelos sklearn (instâncias já inicializadas)
    - X_train, y_train: dados de treino
    - cv: número de folds para cross-validation (padrão: 3)
    - n_jobs: número de threads para paralelismo (-1 usa todos os núcleos)
    
    Retorna:
    - Um dicionário com as métricas de cada modelo
    """
    results = {}

    plt.figure(figsize=(12, 5))
    
    for name, model in dic_models:
        model_name = name
        print(f"\n### Avaliando {model_name} ###")

        # Treina e calcula acurácia com cross-validation
        accuracy_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="accuracy", n_jobs=n_jobs)
        accuracy_mean = accuracy_scores.mean()
        print(f"Acurácia: {accuracy_mean:.4f}")

        # Predições para matriz de confusão
        y_train_pred = cross_val_predict(model, X_train, y_train, cv=cv, n_jobs=n_jobs)

        # Matriz de Confusão
        conf_matrix = confusion_matrix(y_train, y_train_pred)
        print("Matriz de Confusão:\n", conf_matrix)

        # Precision, Recall e F1-score
        precision = precision_score(y_train, y_train_pred)
        recall = recall_score(y_train, y_train_pred)
        f1 = f1_score(y_train, y_train_pred)

        print(f"Precisão: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")

        results[model_name] = {
            "accuracy": accuracy_mean,
            "precision": precision,
            "recall": recall,
            "f1": f1
        }
        
    return results
